<a href="https://colab.research.google.com/github/jwbarbona/labo2025v/blob/main/201_ComparandoModelos_torneo3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparando Modelos

Estamos en la  Clase 02
<br> El objetivo de la materia es lograr la mejor predicción para nuestro probleama, para lo que se probarán pipelines con decenas de alternativas, será indispensable comparar varios modelos predictivos entre sí y decidir cual es el mejor
<br> La tarea no es tan sencilla

## Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


cp: cannot stat '/content/buckets/b1/kaggle/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


# 1  Particion Training/Testing
## Clase 02  Experimento 1

## 1.2 Introduccion

## 1.4 Codigo

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type -> R

limpio el ambiente de R

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657218,35.1,1454494,77.7,1325425,70.8
Vcells,1220160,9.4,8388608,64.0,1975136,15.1


In [ ]:
Sys.time()

[1] "2025-11-22 14:10:40 UTC"

* Instalacion de la libreria  rpart.plot  para dibujar el arbol
* invocacion de las librerias  **data.table** y  **rpart**

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("rpart")


Loading required package: data.table

Loading required package: rpart



###  Accion a Realizar
PARAM$semilla  debe tener su primer semilla aleatoria

In [ ]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "exp201"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))



---



## 2.3  Bibliografía

Demšar, J. [Statistical comparisons of classifiers over multiple data sets](https://www.jmlr.org/papers/volume7/demsar06a/demsar06a.pdf) J. Mach. Learn. Res. 7, 1–30 (2006).   
Starmer, J. [Machine Learning Fundamentals: Cross Validation](https://www.youtube.com/watch?v=fSytzGwwBVw)Machine Learning Fundamentals: Cross Validation, StatQuest with Josh Starmer youtube channel, (2018)
Hastie, T.[The elements of statistical learning: data mining, inference, and prediction](https://hastie.su.domains/Papers/ESLII.pdf) , volume 2. Springer, 2009  ( Chapter 7 Model Assessment and Selection )

## 2.4 Codigo  Montecarlo

El código de la Montecarlo Cross Validation es notablemente más complejo que el anterior de una simple partición training/testing
<br> Se crea la funcion ArbolEstimarGanancia()  que dada una semilla realiza la partición  training/testing, entrena en training, aplica el modelo a testing, y se calculan las ganancias
<br> Se utiliza la funcion **mcmapply**  de R para aplicar ArbolEstimarGanancia a todos los elementos del vector de semillas

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type -> R

limpio el ambiente de R

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

* Instalacion de la libreria  rpart.plot  para dibujar el arbol
* invocacion de las librerias  **data.table** y  **rpart**

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")

###  Accion a Realizar
PARAM$semillas  debe tener sus cinco semillas aleatorias

In [ ]:
PARAM <- list()
PARAM$semillas <- c(102191, 200177, 410551, 552581, 892237)  # aqui debe ir sus CINCO
PARAM$training_pct <- 70L  # entre  1L y 99L

PARAM$rpart <- list (
  "cp" = -1, # complejidad minima
  "minsplit" = 170, # minima cantidad de regs en un nodo para hacer el split
  "minbucket" = 70, # minima cantidad de regs en una hoja
  "maxdepth" = 7 # profundidad máxima del arbol
)


In [ ]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa

# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

particionar <- function(
    data, division, agrupa = "",
    campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}


In [ ]:

ArbolEstimarGanancia <- function(semilla, param_basicos) {
  # particiono estratificadamente el dataset
  particionar(dataset,
    division = c(param_basicos$training_pct, 100L -param_basicos$training_pct),
    agrupa = "clase_ternaria",
    seed = semilla # aqui se usa SU semilla
  )

  # genero el modelo
  # predecir clase_ternaria a partir del resto
  modelo <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_basicos$rpart
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion <- predict(modelo, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad

  # prediccion es una matriz con TRES columnas,
  #  llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  # cada columna es el vector de probabilidades


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test <- dataset[
    fold == 2,
    sum(ifelse(prediccion[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada <- ganancia_test / (( 100 - PARAM$training_pct ) / 100 )

  return(list(
    "semilla" = semilla,
    "testing" = dataset[fold == 2, .N],
    "testing_pos" = dataset[fold == 2 & clase_ternaria == "BAJA+2", .N],
    "envios" = dataset[fold == 2, sum(prediccion[, "BAJA+2"] > 0.025)],
    "aciertos" = dataset[
        fold == 2,
        sum(prediccion[, "BAJA+2"] > 0.025 & clase_ternaria == "BAJA+2")
    ],
    "ganancia_test" = ganancia_test_normalizada
  ))
}


In [ ]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "exp202"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [ ]:

# la funcion mcmapply  llama a la funcion ArbolEstimarGanancia
#  tantas veces como valores tenga el vector  PARAM$semillas
salidas <- mcmapply(ArbolEstimarGanancia,
  PARAM$semillas, # paso el vector de semillas
  MoreArgs = list(PARAM), # aqui paso el segundo parametro
  SIMPLIFY = FALSE,
  mc.cores = detectCores()
)

# muestro la lista de las salidas en testing
#  para la particion realizada con cada semilla
salidas


In [ ]:
# paso la lista a vector
tb_salida <- rbindlist(salidas)
print( tb_salida)

In [ ]:
# finalmente calculo la media (promedio)  de las ganancias
cat( "ganancia promedio: ", tb_salida[, mean(ganancia_test)], "\n" )

### Acciones a realizar
* Reportar la  *Ganancia Promedio* <br> en la planilla colaborativa hoja  **C2-5sem**
* Se discutirá en clase la variabilidad de estos "ganancia promedio de cinco semillas"



---



# 6 Comparando dos distintos  modelos
## Clase 02  Experimento 6

## 6.1  Objetivos
Presentar a los alumnos la dificultad que se presenta ante la comparación de dos modelos, uno posee un poder predictivo claramente mayor al otro.
Comparación simple de media de ganancias, versus probabilidad que un modelo sea superior a otro.

## 6.3 Codigo dos distintos modelos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type -> R

El tiempo de corrida es de alrededor de 40 minutos

limpio el ambiente de R

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

* Instalacion de la libreria  rpart.plot  para dibujar el arbol
* invocacion de las librerias  **data.table** y  **rpart**

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")

if (!require("primes")) install.packages("primes")
require("primes")

require("ggplot2")

###  Accion a Realizar
PARAM$semilla_primigenia  debe reemplazarse por SU primer semilla

In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- 102191
PARAM$qsemillas <- 20
PARAM$training_pct <- 70L     # entre 1 y 100


In [ ]:
# los dos arboles
PARAM$rpart1 <- list (
  "cp" = -1,
  "minsplit" = 170,
  "minbucket" = 70,
  "maxdepth" = 7
)


PARAM$rpart2 <- list (
  "cp" = -1,
  "minsplit" = 250,
  "minbucket" = 125,
  "maxdepth" = 20
)

In [ ]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa

# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

particionar <- function(
    data, division, agrupa = "",
    campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}


In [ ]:
DosArbolesEstimarGanancia <- function(semilla, training_pct, param_rpart1, param_rpart2) {

  # fuerzo impresion
  print( paste( semilla, Sys.time()) )
  flush.console()

  # particiono estratificadamente el dataset
  particionar(dataset,
    division = c(training_pct, 100L -training_pct),
    agrupa = "clase_ternaria",
    seed = semilla # aqui se usa SU semilla
  )

  # genero el modelo
  # predecir clase_ternaria a partir del resto
  modelo1 <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_rpart1
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion1 <- predict(modelo1, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test1 <- dataset[
    fold == 2,
    sum(ifelse(prediccion1[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada1 <- ganancia_test1 / (( 100 - training_pct ) / 100 )

  modelo2 <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_rpart2
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion2 <- predict(modelo2, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test2 <- dataset[
    fold == 2,
    sum(ifelse(prediccion2[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada2 <- ganancia_test2 / (( 100 - training_pct ) / 100 )

  return(list(
    "semilla" = semilla,
    "ganancia1" = ganancia_test_normalizada1,
    "ganancia2" = ganancia_test_normalizada2
  ))
}

In [ ]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "exp206"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [ ]:
detectCores()

In [ ]:
# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo

# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, PARAM$qsemillas )

In [ ]:
# demora interminables 60 minutos en correr en Google Colab
Sys.time()

salidas <- mcmapply( DosArbolesEstimarGanancia,
  PARAM$semillas, # paso el vector de semillas
  MoreArgs = list(PARAM$training_pct, PARAM$rpart1, PARAM$rpart2), # aqui paso el segundo parametro
  SIMPLIFY = FALSE,
  mc.cores = detectCores()
)

Sys.time()

In [ ]:
# paso la lista a vector
tb_salida <- rbindlist(salidas)
tb_salida

In [ ]:
grafico <- ggplot( tb_salida, aes(x=ganancia1)) + geom_density(alpha=0.25, fill="green", color="green")  +
             geom_density(data=tb_salida, aes(x=ganancia2), fill="purple", color="purple",  alpha=0.10)

print(grafico)

In [ ]:
# imprimo en un .pdf  que va a la carpeta del experimento
pdf("densidad_dos.pdf")
print(grafico)
dev.off()

In [ ]:
print( tb_salida[ , list( "arbol1" = mean( ganancia1),  "arbol2" = mean(ganancia2) ) ] )

print( tb_salida[ , list( "prob( m1 > m2)" = sum(ganancia1 > ganancia2 )/ .N ) ]  )

### Acciones a realizar
* Reportar la  los resultados en la planilla colaborativa hoja  **C2-dosModelos**
* ¿Todos los alumnos hubieran elegido el mismo modelo aun con 20 semillas?



---



# 7 Comparando dos buenos  modelos
## Clase 02  Experimento 7

## 7.1  Objetivos
Presentar a los alumnos la dificultad que surge en la comparación de dos muy distintos, pero ambos buenos, modelos predictivos, y el costo computacional asociado a esa comparación
<br>Comparación simple de media de ganancias, versus probabilidad que un modelo sea superior a otro.
<br>Limitaciones de la MonteCarlo Cross Validation
<br>Existencia del Test de Wilcoxon

## 7.2 Introduccion
Estos son los dos arboles de muy distinta profundidad.
<br>¿Cuál de ellos es mejor? ¿Cuál elijo?

| Hiperparámetro | Arbol 1 | Arbol 2 |
| --- | --: |  ---: |
| cp | -1 | -1 |
|minsplit | 170 | 1900 |
|minbucket | 70 | 800 |
|maxdepth | 7 | 3 |



## 7.3 Dos buenos modelos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type -> R

El tiempo de corrida es de alrededor de 50 minutos

limpio el ambiente de R

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

* Instalacion de la libreria  rpart.plot  para dibujar el arbol
* invocacion de las librerias  **data.table** y  **rpart**

## 8.3 Codigo Test de Wilcoxon

In [ ]:
# 1 sola ganancia
wilcox.test(
  c(41470000),
  c(37790000),
  paired = TRUE
)



	Wilcoxon signed rank exact test

data:  c(41470000) and c(37790000)
V = 1, p-value = 1
alternative hypothesis: true location shift is not equal to 0


In [ ]:
# 2 ganancias
wilcox.test(
  c(41470000, 48480000),
  c(37790000, 43660000),
  paired = TRUE
)



	Wilcoxon signed rank exact test

data:  c(41470000, 48480000) and c(37790000, 43660000)
V = 3, p-value = 0.5
alternative hypothesis: true location shift is not equal to 0


In [ ]:
# 3 ganancias
wilcox.test(
  c(41470000, 48480000, 50610000),
  c(37790000, 43660000, 47840000),
  paired = TRUE
)


	Wilcoxon signed rank exact test

data:  c(41470000, 48480000, 50610000) and c(37790000, 43660000, 47840000)
V = 6, p-value = 0.25
alternative hypothesis: true location shift is not equal to 0


In [ ]:
# 4 ganancias
wilcox.test(
  c(41470000, 48480000, 50610000, 45580000),
  c(37790000, 43660000, 47840000, 44490000),
  paired = TRUE
)



	Wilcoxon signed rank exact test

data:  c(41470000, 48480000, 50610000, 45580000) and c(37790000, 43660000, 47840000, 44490000)
V = 10, p-value = 0.125
alternative hypothesis: true location shift is not equal to 0


In [ ]:
# 5 ganancias
wilcox.test(
  c(41470000, 48480000, 50610000, 45580000, 52780000),
  c(37790000, 43660000, 47840000, 44490000, 46750000),
  paired = TRUE
)


	Wilcoxon signed rank exact test

data:  c(41470000, 48480000, 50610000, 45580000, 52780000) and c(37790000, 43660000, 47840000, 44490000, 46750000)
V = 15, p-value = 0.0625
alternative hypothesis: true location shift is not equal to 0


In [ ]:
# 6 ganancias
wilcox.test(
  c(41470000, 48480000, 50610000, 45580000, 52780000, 49970000),
  c(37790000, 43660000, 47840000, 44490000, 46750000, 44300000),
  paired = TRUE
)


	Wilcoxon signed rank exact test

data:  c(41470000, 48480000, 50610000, 45580000, 52780000, 49970000) and c(37790000, 43660000, 47840000, 44490000, 46750000, 44300000)
V = 21, p-value = 0.03125
alternative hypothesis: true location shift is not equal to 0


In [ ]:
# Finalmente el p-value es  0.03125  < 0.05
# hicieron falta  SEIS semillas, seis ganancias, para
#   afirmar que el modelo 1 es superior al modelo 2

# 7 ganancias
wilcox.test(
  c(41470000, 48480000, 50610000, 45580000, 52780000, 49970000, 52810000),
  c(37790000, 43660000, 47840000, 44490000, 46750000, 44300000, 42840000),
  paired = TRUE
)



	Wilcoxon signed rank exact test

data:  c(41470000, 48480000, 50610000, 45580000, 52780000, 49970000, 52810000) and c(37790000, 43660000, 47840000, 44490000, 46750000, 44300000, 42840000)
V = 28, p-value = 0.01563
alternative hypothesis: true location shift is not equal to 0


In [ ]:
# 8 ganancias
wilcox.test(
  c(41470000, 48480000, 50610000, 45580000, 52780000, 49970000, 52810000, 43060000),
  c(37790000, 43660000, 47840000, 44490000, 46750000, 44300000, 42840000, 37300000),
  paired = TRUE
)



	Wilcoxon signed rank exact test

data:  c(41470000, 48480000, 50610000, 45580000, 52780000, 49970000, 52810000, 43060000) and c(37790000, 43660000, 47840000, 44490000, 46750000, 44300000, 42840000, 37300000)
V = 36, p-value = 0.007813
alternative hypothesis: true location shift is not equal to 0


In [ ]:
#


	Wilcoxon signed rank exact test

data:  c(41470000, 48480000, 50610000, 45580000, 52780000, 49970000, 52810000, 43060000, 49660000) and c(37790000, 43660000, 47840000, 44490000, 46750000, 44300000, 42840000, 37300000, 43730000)
V = 45, p-value = 0.003906
alternative hypothesis: true location shift is not equal to 0




---



In [ ]:
# 10 ganancias

PARAM_8HT = c( 6620700, 6579900, 6681900, 5817300, 6032400, 5977500, 6266400, 6620400, 6710700, 6132600)
PARAM_10HT = c( 5873400,6117000,5563200,6168600,6162900,6747900,5963100,6198000,6648000,6473100)
PARAM_12HT = c( 5894100, 6294600, 6810900, 5958300, 6386700, 6668400, 6620400, 6015900, 6340800, 5747700)
PARAM_6HT = c( 5831700,6885600,5992800,6136500,6597600,6172200,6172200,6291300,6216000,5741400)
PARAM_4HT = c( 5903100,6764400,6022500,5929200,5769600,5875500,6233400,5957100,6008100,6402600)
PARAM_2HT = c( 6276600,5625600,6058500,5749200,5953500,6097200,5925900,5591700,6154200,6625500)
PARAM_16HT = c( 5832600,5676900,6025200,6276300,6631500,6075600,5988600,6354000,5962800,5992500)



# para 8HT vs el resto

# ENTRE  ganancias 8HT Y 12HT
wilcox.test(
  PARAM_8HT,
  PARAM_12HT,
  paired = TRUE
)

# ENTRE  ganancias 8HT Y 10HT
wilcox.test(
  PARAM_8HT,
  PARAM_10HT,
  paired = TRUE
)

# ENTRE  ganancias 8HT Y 6HT
wilcox.test(
  PARAM_8HT,
  PARAM_6HT,
  paired = TRUE
)

# ENTRE  ganancias 8HT Y 4HT
wilcox.test(
  PARAM_8HT,
  PARAM_4HT,
  paired = TRUE
)

# ENTRE  ganancias 8HT Y 2HT
wilcox.test(
  PARAM_8HT,
  PARAM_2HT,
  paired = TRUE
)

# ENTRE  ganancias 8HT Y 16HT
wilcox.test(
  PARAM_8HT,
  PARAM_16HT,
  paired = TRUE
)

# para 10HT vs el resto - - - - - - - - - - - - - - - - - -

# ENTRE  ganancias 10HT Y 12HT
print ("ENTRE  ganancias 10HT Y 12HT")
wilcox.test(
  PARAM_10HT,
  PARAM_12HT,
  paired = TRUE
)


# ENTRE  ganancias 10HT Y 6HT
print ("ENTRE  ganancias 10HT Y 6HT")
wilcox.test(
  PARAM_10HT,
  PARAM_6HT,
  paired = TRUE
)

# ENTRE  ganancias 10HT Y 8HT
print ("ENTRE  ganancias 10HT Y 8HT")
wilcox.test(
  PARAM_10HT,
  PARAM_8HT,
  paired = TRUE
)

# ENTRE  ganancias 10HT Y 4HT
print ("ENTRE  ganancias 10HT Y 4HT")
wilcox.test(
  PARAM_10HT,
  PARAM_4HT,
  paired = TRUE
)

# ENTRE  ganancias 10HT Y 2HT
print ("ENTRE  ganancias 10HT Y 2HT")
wilcox.test(
  PARAM_10HT,
  PARAM_2HT,
  paired = TRUE
)

# ENTRE  ganancias 10HT Y 8HT
print ("ENTRE  ganancias 10HT Y 8HT")
wilcox.test(
  PARAM_10HT,
  PARAM_8HT,
  paired = TRUE
)

# ENTRE  ganancias 10HT Y 16HT
print ("ENTRE  ganancias 10HT Y 16HT")
wilcox.test(
  PARAM_10HT,
  PARAM_16HT,
  paired = TRUE
)




	Wilcoxon signed rank exact test

data:  PARAM_8HT and PARAM_12HT
V = 34, p-value = 0.5566
alternative hypothesis: true location shift is not equal to 0



	Wilcoxon signed rank exact test

data:  PARAM_8HT and PARAM_10HT
V = 35, p-value = 0.4922
alternative hypothesis: true location shift is not equal to 0



	Wilcoxon signed rank exact test

data:  PARAM_8HT and PARAM_6HT
V = 38, p-value = 0.3223
alternative hypothesis: true location shift is not equal to 0



	Wilcoxon signed rank exact test

data:  PARAM_8HT and PARAM_4HT
V = 42, p-value = 0.1602
alternative hypothesis: true location shift is not equal to 0



	Wilcoxon signed rank exact test

data:  PARAM_8HT and PARAM_2HT
V = 46, p-value = 0.06445
alternative hypothesis: true location shift is not equal to 0


...


	Wilcoxon signed rank exact test

data:  PARAM_10HT and PARAM_2HT
V = 41, p-value = 0.1934
alternative hypothesis: true location shift is not equal to 0


[1] "ENTRE  ganancias 10HT Y 12HT"



	Wilcoxon signed rank exact test

data:  PARAM_10HT and PARAM_8HT
V = 20, p-value = 0.4922
alternative hypothesis: true location shift is not equal to 0


[1] "ENTRE  ganancias 10HT Y 12HT"



	Wilcoxon signed rank exact test

data:  PARAM_10HT and PARAM_16HT
V = 34, p-value = 0.5566
alternative hypothesis: true location shift is not equal to 0


In [ ]:

# bucle de gemini
# 1. Definición de los Vectores de Datos
PARAM_8HT = c(6620700, 6579900, 6681900, 5817300, 6032400, 5977500, 6266400, 6620400, 6710700, 6132600)
PARAM_10HT = c(5873400, 6117000, 5563200, 6168600, 6162900, 6747900, 5963100, 6198000, 6648000, 6473100)
PARAM_12HT = c(5894100, 6294600, 6810900, 5958300, 6386700, 6668400, 6620400, 6015900, 6340800, 5747700)
PARAM_6HT = c(5831700, 6885600, 5992800, 6136500, 6597600, 6172200, 6172200, 6291300, 6216000, 5741400)
PARAM_4HT = c(5903100, 6764400, 6022500, 5929200, 5769600, 5875500, 6233400, 5957100, 6008100, 6402600)
PARAM_2HT = c(6276600, 5625600, 6058500, 5749200, 5953500, 6097200, 5925900, 5591700, 6154200, 6625500)
PARAM_16HT = c(5832600, 5676900, 6025200, 6276300, 6631500, 6075600, 5988600, 6354000, 5962800, 5992500)

# 2. Definir la lista de vectores a comparar contra la referencia
# Excluimos 8HT de esta lista, ya que es la referencia.
vectores_a_comparar <- list(
  PARAM_10HT = PARAM_10HT,
  PARAM_12HT = PARAM_12HT,
  PARAM_6HT = PARAM_6HT,
  PARAM_4HT = PARAM_4HT,
  PARAM_2HT = PARAM_2HT,
  PARAM_16HT = PARAM_16HT
)

# 3. Inicializar el data frame para almacenar los resultados
resultados_wilcoxon <- data.frame(
  Comparacion = character(),
  N_Pares = numeric(),
  Estadistico_V = numeric(),
  P_Valor = numeric(),
  Significancia_5pc = character(),
  stringsAsFactors = FALSE
)

# 4. Bucle para ejecutar el Test de Wilcoxon para cada comparación
for (nombre in names(vectores_a_comparar)) {
  # Ejecutar el Test de Wilcoxon de Rangos con Signo (muestras pareadas)
  prueba <- wilcox.test(PARAM_8HT, vectores_a_comparar[[nombre]], paired = TRUE, exact = TRUE)

  # Extraer los valores de interés
  p_valor <- prueba$p.value
  estadistico_v <- prueba$statistic
  n_pares <- length(PARAM_8HT)

  # Determinar la significancia
  significancia <- ifelse(p_valor < 0.05, "SI (Rechazar H0)", "NO (Mantener H0)")

  # Crear una nueva fila y añadirla al data frame
  nueva_fila <- data.frame(
    Comparacion = paste0("8HT vs ", sub("PARAM_", "", nombre)),
    N_Pares = n_pares,
    Estadistico_V = estadistico_v,
    P_Valor = p_valor,
    Significancia_5pc = significancia
  )

  resultados_wilcoxon <- rbind(resultados_wilcoxon, nueva_fila)
}

print(resultados_wilcoxon)


   Comparacion N_Pares Estadistico_V    P_Valor Significancia_5pc
V  8HT vs 10HT      10            35 0.49218750  NO (Mantener H0)
V1 8HT vs 12HT      10            34 0.55664062  NO (Mantener H0)
V2  8HT vs 6HT      10            38 0.32226562  NO (Mantener H0)
V3  8HT vs 4HT      10            42 0.16015625  NO (Mantener H0)
V4  8HT vs 2HT      10            46 0.06445312  NO (Mantener H0)
V5 8HT vs 16HT      10            43 0.13085938  NO (Mantener H0)


In [ ]:
# 1. Definición de los Vectores de Datos (Re-definición para asegurar la lista)
PARAM_8HT = c(6620700, 6579900, 6681900, 5817300, 6032400, 5977500, 6266400, 6620400, 6710700, 6132600)
PARAM_10HT = c(5873400, 6117000, 5563200, 6168600, 6162900, 6747900, 5963100, 6198000, 6648000, 6473100)
PARAM_12HT = c(5894100, 6294600, 6810900, 5958300, 6386700, 6668400, 6620400, 6015900, 6340800, 5747700)
PARAM_6HT = c(5831700, 6885600, 5992800, 6136500, 6597600, 6172200, 6172200, 6291300, 6216000, 5741400)
PARAM_4HT = c(5903100, 6764400, 6022500, 5929200, 5769600, 5875500, 6233400, 5957100, 6008100, 6402600)
PARAM_2HT = c(6276600, 5625600, 6058500, 5749200, 5953500, 6097200, 5925900, 5591700, 6154200, 6625500)
PARAM_16HT = c(5832600, 5676900, 6025200, 6276300, 6631500, 6075600, 5988600, 6354000, 5962800, 5992500)

# Lista principal con todos los parámetros
todos_los_parametros <- list(
  PARAM_8HT = PARAM_8HT,
  PARAM_10HT = PARAM_10HT,
  PARAM_12HT = PARAM_12HT,
  PARAM_6HT = PARAM_6HT,
  PARAM_4HT = PARAM_4HT,
  PARAM_2HT = PARAM_2HT,
  PARAM_16HT = PARAM_16HT
)

# 3. Inicializar el data frame para almacenar los resultados
resultados_completos <- data.frame(
  Grupo1 = character(),
  Grupo2 = character(),
  N_Pares = numeric(),
  Estadistico_V = numeric(),
  P_Valor = numeric(),
  Significancia_5pc = character(),
  stringsAsFactors = FALSE
)

# Obtener los nombres de los vectores
nombres <- names(todos_los_parametros)
num_vectores <- length(nombres)
n_pares <- length(todos_los_parametros[[1]]) # Asumimos que todos tienen la misma longitud (10)

# 4. Bucle Doble (Anidado)
# El bucle externo (i) selecciona el primer vector
for (i in 1:(num_vectores - 1)) {
  # El bucle interno (j) selecciona el segundo vector, asegurándose de que j > i
  # Esto evita comparaciones duplicadas (10HT vs 8HT) y comparaciones consigo mismo (8HT vs 8HT)
  for (j in (i + 1):num_vectores) {

    nombre1 <- nombres[i]
    nombre2 <- nombres[j]

    vector1 <- todos_los_parametros[[i]]
    vector2 <- todos_los_parametros[[j]]

    # Ejecutar el Test de Wilcoxon de Rangos con Signo
    prueba <- wilcox.test(vector1, vector2, paired = TRUE, exact = TRUE)

    # Extraer los valores
    p_valor <- prueba$p.value
    estadistico_v <- prueba$statistic

    # Determinar la significancia
    significancia <- ifelse(p_valor < 0.05, "SI (Rechazar H0)", "NO (Mantener H0)")

    # Crear una nueva fila y añadirla
    nueva_fila <- data.frame(
      Grupo1 = sub("PARAM_", "", nombre1),
      Grupo2 = sub("PARAM_", "", nombre2),
      N_Pares = n_pares,
      Estadistico_V = estadistico_v,
      P_Valor = p_valor,
      Significancia_5pc = significancia
    )

    resultados_completos <- rbind(resultados_completos, nueva_fila)
  }
}

# 5. Imprimir y ver la tabla final
print(resultados_completos)
# O usa View(resultados_completos) en Colab/RStudio
# ----------------------------------------------------------------------------------------------------------
# PARA HACER LA TABAL DE DOBLE ENTRADA

 # 1. Instalar el paquete (Solo necesitas hacer esto una vez)
install.packages("tidyverse")

# 2. Cargar el paquete en tu sesión de R (NECESARIO antes de usar la función)
library(tidyverse)

# 1. Ejecutar el código de comparaciones (asumido que ya se corrió y 'resultados_completos' existe)
# (Si no lo tienes, usa el código de la respuesta anterior para generarlo)

# Asumiendo que 'resultados_completos' tiene las columnas: Grupo1, Grupo2, P_Valor

# Instalar y cargar el paquete tidyr (si no lo tienes instalado)
# install.packages("tidyr")
library(tidyr)
library(dplyr) # Necesario para algunas manipulaciones (aunque 'pivot_wider' es de 'tidyr')

# 2. Transformar el data frame largo a formato ancho (matriz asimétrica)
matriz_p_valores_raw <- resultados_completos %>%
  # Seleccionar solo las columnas necesarias
  select(Grupo1, Grupo2, P_Valor) %>%
  # Reformatear a ancho, usando Grupo1 como filas y Grupo2 como columnas
  pivot_wider(
    names_from = Grupo2,    # Valores de la columna Grupo2 serán los nombres de las nuevas columnas
    values_from = P_Valor  # Los valores de P_Valor llenarán la matriz
  )

# 3. Construcción de la Matriz Simétrica y Limpieza
# Necesitamos una matriz que incluya todos los nombres de los parámetros (2HT, 4HT, etc.)

# Obtener todos los nombres únicos de los grupos
nombres_grupos <- unique(c(resultados_completos$Grupo1, resultados_completos$Grupo2))
nombres_grupos <- sort(nombres_grupos) # Opcional: ordenar los nombres

# Convertir el data frame a una matriz e incluir los nombres faltantes (para simetría)
# Primero, hacemos que los nombres de las filas sean los grupos
matriz_base <- matriz_p_valores_raw %>%
  column_to_rownames(var = "Grupo1")

# Crear una matriz vacía para rellenar con los valores
num_grupos <- length(nombres_grupos)
matriz_final <- matrix(NA, nrow = num_grupos, ncol = num_grupos, dimnames = list(nombres_grupos, nombres_grupos))

# 4. Rellenar la Matriz Simétricamente
# Rellenar los valores superiores con los resultados de 'matriz_base'
for (r in rownames(matriz_base)) {
  for (c in colnames(matriz_base)) {
    matriz_final[r, c] <- matriz_base[r, c]
  }
}

# Rellenar la mitad inferior (simetría: p(A vs B) = p(B vs A))
for (i in 1:num_grupos) {
  for (j in 1:num_grupos) {
    if (i > j) { # Solo para la mitad inferior
      matriz_final[i, j] <- matriz_final[j, i]
    }
  }
}

# 5. Visualizar la Matriz final
print("Matriz de P-Valores de Wilcoxon:")
print(matriz_final)

Warning message in wilcox.test.default(vector1, vector2, paired = TRUE, exact = TRUE):
“cannot compute exact p-value with ties”


    Grupo1 Grupo2 N_Pares Estadistico_V    P_Valor Significancia_5pc
V      8HT   10HT      10            35 0.49218750  NO (Mantener H0)
V1     8HT   12HT      10            34 0.55664062  NO (Mantener H0)
V2     8HT    6HT      10            38 0.32226562  NO (Mantener H0)
V3     8HT    4HT      10            42 0.16015625  NO (Mantener H0)
V4     8HT    2HT      10            46 0.06445312  NO (Mantener H0)
V5     8HT   16HT      10            43 0.13085938  NO (Mantener H0)
V6    10HT   12HT      10            27 1.00000000  NO (Mantener H0)
V7    10HT    6HT      10            26 0.92187500  NO (Mantener H0)
V8    10HT    4HT      10            33 0.62500000  NO (Mantener H0)
V9    10HT    2HT      10            41 0.19335938  NO (Mantener H0)
V10   10HT   16HT      10            34 0.55664062  NO (Mantener H0)
V11   12HT    6HT      10            31 0.76953125  NO (Mantener H0)
V12   12HT    4HT      10            40 0.23242188  NO (Mantener H0)
V13   12HT    2HT      10         

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.1     ✔ readr     2.1.6
✔ ggplot2   4.0.1     ✔ stringr   1.6.0
✔ lubridate 1.9.4     ✔ tibble    3.3.0
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[1] "Matriz de P-Valores de Wilcoxon:"
          10HT      12HT      16HT       2HT       4HT       6HT 8HT
10HT        NA 1.0000000 0.5566406 0.1933594 0.6250000 0.9218750  NA
12HT 1.0000000        NA 0.1848583 0.1601562 0.2324219 0.7695312  NA
16HT 0.5566406 0.1848583        NA        NA        NA        NA  NA
2HT  0.1933594 0.1601562        NA        NA        NA        NA  NA
4HT  0.6250000 0.2324219        NA        NA        NA        NA  NA
6HT  0.9218750 0.7695312        NA        NA        NA        NA  NA
8HT         NA        NA        NA        NA        NA        NA  NA


# 9 Comparando automaticamente modelos con test de Wilcoxon
## Clase 02  Experimento 9

## 9.1  Objetivos

Proveer un script que permita comparar dos arboles de decisión con la librería rpart utilizando la menor cantidad de cómputo que permita el Test de Wilxcoxon

## 9.2  Introduccion

| Hiperparámetro | Arbol 1 | Arbol 2 |
| --- | --- |  --- |
| cp | -1 | -1 |
|minsplit | 1050 | 650 |
|minbucket | 550 | 300 |
|maxdepth |67 | 6 |




## 9.3 Codigo comparacion automatica Wilcoxon

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type -> R

limpio el ambiente de R

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

* Instalacion de la libreria  rpart.plot  para dibujar el arbol
* invocacion de las librerias  **data.table** y  **rpart**

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")

if (!require("primes")) install.packages("primes")
require("primes")

require("ggplot2")

###  Accion a Realizar
PARAM$semilla_primigenia  debe reemplazarse por SU primer semilla

In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- 102191
PARAM$qsemillas_tope <- 50
PARAM$training_pct <- 70L     # entre 1 y 100


In [ ]:
# los dos arboles
PARAM$rpart1 <- list (
  "cp" = -1,
  "minsplit" = 800,
  "minbucket" = 400,
  "maxdepth" = 7
)


PARAM$rpart2 <- list (
  "cp" = -1,
  "minsplit" = 650,
  "minbucket" = 300,
  "maxdepth" = 6
)


In [ ]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa

# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

particionar <- function(
    data, division, agrupa = "",
    campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}


In [ ]:
DosArbolesEstimarGanancia <- function(semilla, training_pct, param_rpart1, param_rpart2) {
  # particiono estratificadamente el dataset
  particionar(dataset,
    division = c(training_pct, 100L -training_pct),
    agrupa = "clase_ternaria",
    seed = semilla # aqui se usa SU semilla
  )

  # genero el modelo
  # predecir clase_ternaria a partir del resto
  modelo1 <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_rpart1
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion1 <- predict(modelo1, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test1 <- dataset[
    fold == 2,
    sum(ifelse(prediccion1[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada1 <- ganancia_test1 / (( 100 - training_pct ) / 100 )

  modelo2 <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_rpart2
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion2 <- predict(modelo2, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test2 <- dataset[
    fold == 2,
    sum(ifelse(prediccion2[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada2 <- ganancia_test2 / (( 100 - training_pct ) / 100 )

  return(list(
    "semilla" = semilla,
    "ganancia1" = ganancia_test_normalizada1,
    "ganancia2" = ganancia_test_normalizada2
  ))
}

In [ ]:
# 1  ->  el modelo 1 es mejor
# 2  ->  el modelo 2 es mejor
# 0  ->  No se pudo determinar con el tope de qsemillas_tope


MejorArbol <- function( qsemillas_tope, training_pct, param_rpart1, param_rpart2) {

  # genero numeros primos
  primos <- generate_primes(min = 100000, max = 1000000)
  set.seed(PARAM$semilla_primigenia) # inicializo
  # me quedo con PARAM$qsemillas   semillas
  semillas <- sample(primos, qsemillas_tope )

  pvalue <- 1.0
  isem <- 1
  vgan1 <- c() # almaceno ganancias del modelo1
  vgan2 <- c() # almaceno ganancias del modelo2

  while( (isem <= qsemillas_tope)  & (pvalue > 0.05) ) {

    res <- DosArbolesEstimarGanancia(
       semillas[ isem ],
       training_pct,
       param_rpart1,
       param_rpart2
    )

    vgan1 <- c( vgan1, res$ganancia1 )
    vgan2 <- c( vgan2, res$ganancia2 )

    wt <- wilcox.test( vgan1, vgan2, paired=TRUE )
    pvalue <- wt$p.value

    cat( isem, res$ganancia1, res$ganancia2, pvalue, "\n" )
    flush.console()
    isem <- isem + 1
  }

  out <- 0

  if( pvalue < 0.05 & mean(vgan1) > mean(vgan2)  )  out <- 1
  if( pvalue < 0.05 & mean(vgan1) < mean(vgan2)  )  out <- 2


  return( list( "out" = out,
    "qsemillas" = length(vgan1),
    "m1" = mean( vgan1 ),
    "m2" = mean( vgan2 )
   ) )
}


In [ ]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "exp209"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [ ]:
Sys.time()

comparacion <- MejorArbol(
   PARAM$qsemillas_tope,
   PARAM$training_pct,
   PARAM$rpart1,
   PARAM$rpart2
 )


print( comparacion )

Sys.time()

### Acciones a realizar
* Reportar la  los resultados en la planilla colaborativa hoja  **C2-Wilcox**




---

